In [1]:
import numpy as np
import pandas as pd
import json
import pickle
import matplotlib.pyplot as plt
import warnings

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch.utils.data

import matplotlib.pyplot as plt
import os

In [2]:
'''
Read in saved dataset to use to feed into the model.
Further pre-processing still needs to be done on the state vectors
'''

data_df = pd.read_pickle('/home/amans/Development/scott/nfl-big-data-bowl-2022/clean_dir/ball_carrier_data.pkl')


data_df.head()

,season,gameId,playId,playIndex,timeIndex,time,playerId,football_pos,state,next_state,reward,action,next_action
0,2018,2018090600,677,0,0,2018-09-07T01:34:34.500,44979.0,"[[11.15, 42.28, 0.17, 2.91, 6.78, 0.0, 0.0, 11...","[[11.53, 42.45, 0.14, 1.32, 3.03, 117.3, 148.0...","[[11.62, 42.32, 0.16, 1.7, 3.13, 114.42, 144.8...",[-0.09],"[[-0.09, 0.13, -0.02, -0.38, -0.1, 2.88, 3.22,...","[[-0.16, 0.16, -0.07, -0.53, -0.24, -17.77, 7...."
1,2018,2018090600,677,0,1,2018-09-07T01:34:34.600,44979.0,"[[11.51, 42.1, 0.39, 1.64, 4.44, 0.0, 0.0, 11....","[[11.62, 42.32, 0.16, 1.7, 3.13, 114.42, 144.8...","[[11.78, 42.16, 0.23, 2.23, 3.37, 132.19, 137....",[-0.16],"[[-0.16, 0.16, -0.07, -0.53, -0.24, -17.77, 7....","[[-0.17, 0.18, -0.01, -0.4, -0.51, 6.53, 2.66,..."
2,2018,2018090600,677,0,2,2018-09-07T01:34:34.700,44979.0,"[[11.47, 42.06, 0.06, 1.2, 3.4, 0.0, 0.0, 11.4...","[[11.78, 42.16, 0.23, 2.23, 3.37, 132.19, 137....","[[11.95, 41.98, 0.24, 2.63, 3.88, 125.66, 134....",[-0.17],"[[-0.17, 0.18, -0.01, -0.4, -0.51, 6.53, 2.66,...","[[-0.21, 0.19, -0.05, -0.48, -0.22, 4.32, 3.47..."
3,2018,2018090600,677,0,3,2018-09-07T01:34:34.800,44979.0,"[[12.39, 41.68, 1.0, 0.93, 2.17, 0.0, 0.0, 12....","[[11.95, 41.98, 0.24, 2.63, 3.88, 125.66, 134....","[[12.16, 41.79, 0.29, 3.11, 4.1, 121.34, 131.2...",[-0.21],"[[-0.21, 0.19, -0.05, -0.48, -0.22, 4.32, 3.47...","[[-0.25, 0.21, -0.03, -0.44, -0.67, 2.49, 3.83..."
4,2018,2018090600,677,0,4,2018-09-07T01:34:34.900,44979.0,"[[13.5, 41.18, 1.22, 1.47, 3.5, 0.0, 0.0, 13.5...","[[12.16, 41.79, 0.29, 3.11, 4.1, 121.34, 131.2...","[[12.41, 41.58, 0.32, 3.55, 4.77, 118.85, 127....",[-0.25],"[[-0.25, 0.21, -0.03, -0.44, -0.67, 2.49, 3.83...","[[-0.32, 0.21, -0.06, -0.47, -0.03, 0.0, 9.25,..."


In [3]:
'''
State vector 
['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', \
'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']
'''

data_df.loc[0,'state'][0]

array([ 1.15300000e+01,  4.24500000e+01,  1.40000000e-01,  1.32000000e+00,
        3.03000000e+00,  1.17300000e+02,  1.48060000e+02,  1.15300000e+01,
        1.58000000e+01,  2.73000000e+01,  5.80600000e+01,  8.27327901e-01,
       -5.61719276e-01,  9.98232532e-01,  5.94290537e-02,  0.00000000e+00,
        1.02310000e+01,  3.04200000e+01])

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


In [5]:
'''
Create torch dataset
'''

features = torch.zeros(len(data_df)).to(device)
labels = torch.zeros(len(data_df)).to(device)

print("size = ", features.size())

size =  torch.Size([192049])


In [7]:
'''
Function that splits the data into a training, validation, and test set
'''
def split_data(dataset, train_split, seed):
    np.random.seed(seed)
    indices = list(range(len(dataset)))
    np.random.shuffle(indices)

    train_num = int(len(dataset)*train_split)
    val_num = (len(dataset) - int(len(dataset)*train_split))//2

    train_indices = indices[0:train_num]
    val_indices = indices[train_num:train_num+val_num]
    test_indices = indices[train_num+val_num:]

    #check to make sure slices correct
    assert len(dataset) == len(train_indices) + len(val_indices) + len(test_indices)

    #dataset = help.normalize(train_indices, dataset)

    train_data = dataset.iloc[train_indices,:]
    val_data = dataset.iloc[val_indices,:]
    test_data = dataset.iloc[test_indices,:]

    return train_data, val_data, test_data

train_data, val_data, test_data = split_data(data_df, 0.7, 2430)

print(test_data.columns)

print(f"Length of training data: {len(train_data)}")

Index(['season', 'gameId', 'playId', 'playIndex', 'timeIndex', 'time',
       'playerId', 'football_pos', 'state', 'next_state', 'reward', 'action',
       'next_action'],
      dtype='object')
Length of training data: 134434


In [8]:
'''
Predict ['s', 'a', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir']
from 
['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', \
'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']
'''



'''
Define hyperparamters
'''
BATCH_SIZE = 32
state_size = len(data_df.loc[0,'state'][0])
action_size = 8
gamma = 0.99
#max_action = torch.tensor(max_action, dtype=torch.float32).to(device=device)

print(f"State size: {state_size}")
print(f"Action size: {action_size}")
    
'''
First CNN model
cnn_model = nn.Sequential(
    nn.Conv2d(1, 12, (12,1), stride=1, padding=0),
    nn.LeakyReLU(),
    nn.Conv2d(12, 24, (6, 1)),
    nn.LeakyReLU(),
    nn.Conv2d(24, 48, (1,1)),
    nn.Flatten(),
    nn.Linear(48*6*6,action_size)
)

cnn_model.to(device=device)
'''

State size: 18
Action size: 8


'\nFirst CNN model\ncnn_model = nn.Sequential(\n    nn.Conv2d(1, 12, (12,1), stride=1, padding=0),\n    nn.LeakyReLU(),\n    nn.Conv2d(12, 24, (6, 1)),\n    nn.LeakyReLU(),\n    nn.Conv2d(24, 48, (1,1)),\n    nn.Flatten(),\n    nn.Linear(48*6*6,action_size)\n)\n\ncnn_model.to(device=device)\n'

In [9]:
'''
Try model predicting entire sequence
'''

'\nTry model predicting entire sequence\n'

In [107]:
'''
same results, not great val loss and then overfit
'''
# cnn_model = nn.Sequential(
#     nn.Conv2d(1, 32, (3,3), stride=1,padding=0),
#     nn.LeakyReLU(),
#     #nn.MaxPool2d((2,2), stride=2),
#     nn.Conv2d(32, 64, (3,3), stride=1,padding=0),
#     nn.LeakyReLU(),
#     #nn.MaxPool2d((2,2), stride=2),
#     nn.Conv2d(64,128, (2,2), stride=1,padding=0),
#     nn.LeakyReLU(),
#     #nn.MaxPool2d((2,2), stride=2),
#     nn.Conv2d(128,256, (1,1), stride=1,padding=0),
#     nn.LeakyReLU(),
#     #nn.MaxPool2d((2,2), stride=2),
#     #nn.Conv2d(256,512, (3,3), stride=1,padding=0),
#     #nn.LeakyReLU(),
#     #nn.MaxPool2d((2,2), stride=2),
#     nn.Flatten(),
#     nn.Linear(13824, 6000),
#     nn.LeakyReLU(),
#     nn.Linear(6000, 2000),
#     nn.LeakyReLU(),
#     nn.Linear(2000, 500),
#     nn.LeakyReLU(),
#     nn.Linear(500, action_size*23)
# )

'''
val loss @ .3 after 4 epochs, .287 after 7 epochs, .286 after 8 epochs
'''
# cnn_model = nn.Sequential(
#     nn.Conv2d(1, 6, (2,2), stride=1, padding=0),
#     nn.Flatten(),
#     nn.Linear(924,400), 
#     nn.LeakyReLU(),
#     nn.Linear(400,action_size*23)
# )

'''
Similar val losses. .359 val loss after 3 epochs, .313 after 4
'''
# cnn_model = nn.Sequential(
#     nn.Conv2d(1, 3, (2,2), stride=1, padding=0),
#     nn.Flatten(),
#     nn.Linear(462,230), 
#     nn.LeakyReLU(),
#     nn.Linear(230,action_size*23)
# )

'''
Similar val losses, .283 val loss after 2 epochs, .281 after 4
'''
# cnn_model = nn.Sequential(
#     nn.BatchNorm2d(1),
#     nn.Conv2d(1, 32, (8,8), stride=1, padding=0),
#     nn.Flatten(),
#     nn.Linear(128*4,230), 
#     nn.LeakyReLU(),
#     nn.Linear(230,action_size*23)
# )


'''
Predicting next state
'''
'''
class cnnModel(nn.Module):
    def __init__(self, action_size):
        super(cnnModel, self).__init__()
        
        self.action_size = action_size
        
        self.batch_norm = nn.BatchNorm1d(8)
        self.conv1 = nn.Conv2d(1, 32, (8,8), stride=1, padding=0)
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(128*4,230)
        self.leaky = nn.LeakyReLU()
        self.linear2 = nn.Linear(230,action_size*23)
        
        self.drop = nn.Dropout(.5)
        
    # input (batch_size, 1, 23, action_size)
    def forward(self, input_tensor):
        
        #tens_reshape = input_tensor.view(-1, 23, 8)
        initial_shape = input_tensor.shape
        
        batched_input = self.batch_norm(torch.transpose(input_tensor.view(-1, 23, 8), 1,2))
        
        inp = torch.transpose(batched_input, 1, 2).view(initial_shape)
        
        output = self.conv1(inp)
        output = self.flatten(output)
        output = self.drop(output)
        output = self.linear1(output)
        output = self.leaky(output)
        output = self.linear2(output)
        
        return output
cnn_model = cnnModel(action_size)
'''

# after 50 epochs, 0.3251 val loss with small dataset model

cnn_model = nn.Sequential(
    nn.Linear(185,230), 
    nn.LeakyReLU(),
    nn.Linear(230,184)
)

cnn_model.to(device)

Sequential(
  (0): Linear(in_features=185, out_features=230, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=230, out_features=184, bias=True)
)

In [108]:
from torch.optim.lr_scheduler import LambdaLR
from torch.optim.lr_scheduler import ExponentialLR

#cnn_model.load_state_dict(torch.load("full_dataset_model.pt"))

# define loss function
mse_loss_fn = nn.MSELoss()
#mse_loss_fn = nn.L1Loss()
# define optimizers
cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=0.005)

#scheduler = ExponentialLR(cnn_optimizer, gamma=0.9)

In [109]:
test_play = train_data.iloc[[2,3,4],:]

ids = np.array(test_play.loc[:,'playerId']).reshape(3,1)

print(ids)

np.append(np.zeros((3,2)), ids, axis=1).shape

[[42412.]
 [48988.]
 [46309.]]


(3, 3)

In [110]:
'''
Given indices, train model on batch
['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', \
'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']

Try adding player id as feature
'''
def train_loop_large(df, indices, model, loss_fn, optimizer, device):
    size = len(indices)
    play = df.iloc[indices, :]
    
    numpy_state = np.stack(play['state'].values)
    batch_size = numpy_state.shape[0]
    playerId = np.array(play.loc[:,'playerId']).reshape(batch_size, 1)

    numpy_reshaped_state = np.stack(play['state'].values)[:,:,[3,4,7,8,11,12,13,14]].reshape(batch_size, -1)
    numpy_with_playerid = np.append(numpy_reshaped_state, playerId, axis=1)
    true_state = torch.tensor(numpy_with_playerid, dtype=torch.float32).to(device=device)
    
    numpy_reshaped_next_state = np.stack(play['next_state'].values)[:,:,[3,4,7,8,11,12,13,14]].reshape(batch_size, -1)
    #numpy_next_with_playerid = np.append(numpy_reshaped_next_state, playerId, axis=1)
    true_next_state = torch.tensor(numpy_reshaped_next_state, dtype=torch.float32).to(device=device)
    
    # shape true state into matrix to pass into CNN
    #true_state_mod = torch.tensor(numpy_state_input, dtype=torch.float32).to(device=device)

    model.train()  # put model to training mode

    #compute prediction and loss
    predicted_next_state = model(true_state)
    
    loss = loss_fn(predicted_next_state, true_next_state)

    #Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss/size

In [111]:
def test_loop_large(test_df, model, loss_fn, device):
    size = len(test_df)
    test_loss = 0
    
    with torch.no_grad():
        for row_index in range(0,len(test_df)):

            play = test_df.iloc[row_index, :]

            numpy_state = np.stack(play['state'])
            batch_size = 1
            playerId = np.array(play.loc['playerId']).reshape(batch_size, 1)
            
            numpy_reshaped_state = np.stack(play['state'])[:,[3,4,7,8,11,12,13,14]].reshape(batch_size, -1)
            numpy_with_playerid = np.append(numpy_reshaped_state, playerId, axis=1)
            true_state = torch.tensor(numpy_with_playerid, dtype=torch.float32).to(device=device)

            numpy_reshaped_next_state = np.stack(play['next_state'])[:,[3,4,7,8,11,12,13,14]].reshape(batch_size, -1)
            #numpy_next_with_playerid = np.append(numpy_reshaped_next_state, playerId, axis=1)
            true_next_state = torch.tensor(numpy_reshaped_next_state, dtype=torch.float32).to(device=device)
            
            model.eval()  # put model to eval mode

            #compute prediction and loss
            predicted_next_state = model(true_state)
  
            loss = loss_fn(predicted_next_state, true_next_state)
            
            test_loss += loss
        
        test_loss = test_loss/size

    
    return test_loss

In [112]:
import random

epochs = 10
training_loss_list = []
val_loss_list = []

num_batches = int(len(train_data)/BATCH_SIZE)+1

print(f"Num batches: {num_batches}")

indexes = list(range(0,len(train_data)))

np.random.seed(2430)

for k in range(epochs):
    
    random.shuffle(indexes)
    
    for i in range(num_batches):

        start_index = i*BATCH_SIZE
        end_index = min(len(train_data), (i+1)*BATCH_SIZE)
        
        #indices = list(range(start_index, end_index))
        indices = indexes[start_index:end_index]

        train_loss = train_loop_large(train_data, indices, cnn_model, mse_loss_fn, cnn_optimizer, device)
        
        training_loss_list.append(train_loss)

        if (i % int(num_batches/3) == 0):
        #if (i % int(num_batches) == 0):# and (i != 0):
            #for g in cnn_optimizer.param_groups:
                #g['lr'] = 0.001
                #print(g['lr'])

            val_loss = test_loop_large(val_data, cnn_model, F.mse_loss, device)
            val_loss_list.append(val_loss)
            print(f"At epoch {k}, iter {i}: train loss = {train_loss}")
            print(f"At epoch {k}, iter {i}: val loss = {val_loss}")
    
    for g in cnn_optimizer.param_groups:
        g['lr'] /= 2
        print('post scheduler step lr = ', g['lr'])
            
    #if k > 20 and k % 5 == 0:
    #    for g in cnn_optimizer.param_groups:
    #        #g['lr'] = 0.001
    #        g['lr'] /= 2
    #        print('post scheduler step lr = ', g['lr'])

    #scheduler.step()
            


Num batches: 4202
At epoch 0, iter 0: train loss = 20935.59765625
At epoch 0, iter 0: val loss = 259201.59375
At epoch 0, iter 1400: train loss = 0.2765701711177826
At epoch 0, iter 1400: val loss = 8.051009178161621
At epoch 0, iter 2800: train loss = 0.1845938265323639
At epoch 0, iter 2800: val loss = 6.21742582321167
At epoch 0, iter 4200: train loss = 0.21624422073364258
At epoch 0, iter 4200: val loss = 6.234797954559326
post scheduler step lr =  0.0025
At epoch 1, iter 0: train loss = 0.2914848327636719
At epoch 1, iter 0: val loss = 6.036000728607178


KeyboardInterrupt: 

In [88]:
#torch.save(cnn_model.state_dict(), 'full_dataset_model.pt')

In [22]:
def test_baseline(test_df, loss_fn):
    size = len(test_df)
    test_loss = 0

    for row_index in range(0,len(test_df)):
        
        play = test_df.iloc[row_index, :]

        true_state = torch.tensor(play['state'][:,[3,4,7,8,11,12,13,14]].reshape(-1), dtype=torch.float32)
        true_next_state = torch.tensor(play['next_state'][:,[3,4,7,8,11,12,13,14]].reshape(-1), dtype=torch.float32)

        loss = loss_fn(true_next_state, true_state)

        test_loss += loss

    test_loss = test_loss/size
    
    return test_loss

val_loss = test_baseline(val_data, F.mse_loss)
print('val loss ', val_loss)

val loss  tensor(0.5202)
